<a href="https://colab.research.google.com/github/MarVinikoi/DL_NafldReserch/blob/main/ImageClassif_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tcn
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 33.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Reinitialized existing Git repository in /content/.git/
[master (root-commit) 7455852] first commit
 1 file changed, 2 insertions(+)
 create mode 100644 README.md
fatal: remote origin already exists.
fatal: could not read Username for 'https://github.com': No such device or address


In [2]:
from tcn import TCN, tcn_full_summary
from tensorflow.python.keras.models import Sequential
import cv2
import tensorflow as tf
import numpy as np
import os
from sklearn.model_selection import train_test_split
import math
import random
from google.colab.patches import cv2_imshow


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Variáveis Globais

path_doentes = "/content/drive/MyDrive/PesquisaNAFLD/Data/NafldClassificationImage/images/doentes"
path_saudaveis = "/content/drive/MyDrive/PesquisaNAFLD/Data/NafldClassificationImage/images/nao_doentes"

In [4]:

def dataAugmentation(on = False):
  if on:
    for paciente in os.listdir(path_doentes):
      for img in os.listdir(os.path.join(path_doentes, paciente)):
        image_path = os.path.join(path_doentes,os.path.join(paciente, img))
        image = cv2.imread(image_path)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        vflip_image = cv2.flip(image, 0)
        hflip_image = cv2.flip(image, 1)
        vflip_image_name = nameFormatter(0, img, "Vflip.jpg", paciente) 
        hflip_image_name = nameFormatter(0, img, "Hflip.jpg", paciente)
        #cv2_imshow(image)
        #print(vflip_image_name)
        #print(hflip_image_name)
        cv2.imwrite(vflip_image_name, vflip_image)
        cv2.imwrite(hflip_image_name, hflip_image)


    for paciente in os.listdir(path_saudaveis):
      for img in os.listdir(os.path.join(path_saudaveis, paciente)):
        image_path = os.path.join(path_saudaveis, os.path.join(paciente, img))
        image = cv2.imread(image_path)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        vflip_image = cv2.flip(image, 0)
        hflip_image = cv2.flip(image, 1)
        vflip_image_name = nameFormatter(1, img, "Vflip.jpg", paciente) 
        hflip_image_name = nameFormatter(1, img, "Hflip.jpg", paciente)
        #cv2_imshow(image)
        #print(vflip_image_name)
        #print(hflip_image_name)
        cv2.imwrite(vflip_image_name, vflip_image)
        cv2.imwrite(hflip_image_name, hflip_image)
        

def nameFormatter( path_opt, string1 = "", string2 = "", string3 = ""):
  '''
  @param path_opt - 0 para o path de pacientes doentes e qualquer outro valor para saudáveis
  @param string1 - nome do arquivo que foi alterado nas operações de Data Aug
  @param string2 - string que identifica a operação realizada na imagem
  @param string3 - paciente atual cuja imagem modificada pertence
  '''

  dot_index = string1.find(".")
  string1 = string1[:dot_index]
  string1 = string1 + string2
  if path_opt == 0:
    filename = os.path.join(os.path.join(path_doentes, string3), string1)
  else:
    filename = os.path.join(os.path.join(path_saudaveis, string3), string1)
  return filename


In [6]:
def main():
  dataAugmentation(False) #Só executa com o parametro setado em True
  loadData()
  

if __name__ == "__main__":
  main()

(14, 3, 40000)
(22, 3, 40000)
9
14
11
18
Labels de Treino: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Labels de Teste: [0, 0, 1, 1, 1, 1]
Labels de Validação: [1, 1, 1, 1, 1, 1, 1]
(23, 3, 40000)
(6, 3, 40000)
(7, 3, 40000)



Epoch 1/100
1/1 [==============================] - 11s 11s/step - loss: 263.1364 - accuracy: 0.3913 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 64ms/step - loss: 6239.4580 - accuracy: 0.6087 - val_loss: 981.9450 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 55ms/step - loss: 4653.1323 - accuracy: 0.3913 - val_loss: 3413.7827 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 55ms/step - loss: 2842.8679 - accuracy: 0.4348 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 55ms/step - loss: 2582.7725 - accuracy: 0.5652 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6

In [5]:


def loadData():
  
  labels = []
  pacientes = []
  doentes = []
  saudaveis = []
  labels_doentes = []
  labels_saudaveis = []
  
  treino = []
  teste = []
  val = []

  labels_treino = []
  labels_teste = []
  labels_val = []

  doentes_size = 91
  saudaveis_size = 132
  all_size = 223

  vetor_imgs = []
  vetor_final_imgs_doentes = []
  vetor_final_imgs_saudaveis = []

  #Carregando os vetores de imagens e os transformando em séries temporais
  #Vetor de Doentes
  for paciente in os.listdir(path_doentes):
    for i, img in enumerate(os.listdir(os.path.join(path_doentes, paciente))):
      num_imgs_folder = len(os.listdir(os.path.join(path_doentes, paciente)))
      #Se a imagem for a primeira, última ou do meio, iremos operar sobre ela
      #if( i == 0 or i == num_imgs_folder-1 or i == math.floor((num_imgs_folder-1)/2)):
      if i < 3:  
        path_img = os.path.join(os.path.join(path_doentes, paciente), img)
        image = cv2.imread(path_img)
        r_img, g_img, b_img = cv2.split(image)
        
        r_img = np.array(r_img)
        r_img = np.ndarray.flatten(r_img)

        g_img = np.array(g_img)
        g_img = np.ndarray.flatten(g_img)

        b_img = np.array(b_img)
        b_img = np.ndarray.flatten(b_img)

        #image_array = np.concatenate((r_img, g_img, b_img))
        vetor_imgs.append(r_img)
    
    vetor_final_imgs_doentes.append((vetor_imgs[0], vetor_imgs[1], vetor_imgs[2]))
    vetor_imgs.clear()
  vetor_final_imgs_doentes = np.array(vetor_final_imgs_doentes)  
  print(vetor_final_imgs_doentes.shape)
  


  #Vetor de Saudáveis
  for paciente in os.listdir(path_saudaveis):
    for i, img in enumerate(os.listdir(os.path.join(path_saudaveis, paciente))):
      num_imgs_folder = len(os.listdir(os.path.join(path_saudaveis, paciente)))
      #Se a imagem for a primeira, última ou do meio, iremos operar sobre ela
      #if( i == 0 or i == num_imgs_folder-1 or i == math.floor((num_imgs_folder-1)/2)):
      if i < 3:
        path_img = os.path.join(os.path.join(path_saudaveis, paciente), img)
        image = cv2.imread(path_img)
        r_img, g_img, b_img = cv2.split(image)
        
        r_img = np.array(r_img)
        r_img = np.ndarray.flatten(r_img)
        
        '''
        g_img = np.array(g_img)
        g_img = np.ndarray.flatten(g_img)

        b_img = np.array(b_img)
        b_img = np.ndarray.flatten(b_img)

        image_array = np.concatenate((r_img,g_img, b_img))
        '''      
        vetor_imgs.append(r_img)
    vetor_final_imgs_saudaveis.append((vetor_imgs[0], vetor_imgs[1], vetor_imgs[2]))
    vetor_imgs.clear()

  vetor_final_imgs_saudaveis = np.array(vetor_final_imgs_saudaveis)  
  print(vetor_final_imgs_saudaveis.shape)

  n_imgs_treino_doe = math.floor((len(vetor_final_imgs_doentes)*0.65))
  n_imgs_treino_sau = math.floor((len(vetor_final_imgs_saudaveis)*0.65))
  print(n_imgs_treino_doe)
  print(n_imgs_treino_sau)

  n_imgs_test_doe = n_imgs_treino_doe + math.floor((len(vetor_final_imgs_doentes)*0.2))
  n_imgs_test_sau = n_imgs_treino_sau + math.floor((len(vetor_final_imgs_saudaveis)*0.2))
  print(n_imgs_test_doe)
  print(n_imgs_test_sau)

  #n_imgs_val_doe = n_imgs_test_doe + math.floor((len(vetor_final_imgs_doentes)*0.15))
  #n_imgs_val_sau = n_imgs_test_sau + math.floor((len(vetor_final_imgs_saudaveis)*0.15))

  #Utilizando cerca de 65% das imagens para treino
  treino.extend(vetor_final_imgs_doentes[:n_imgs_treino_doe])
  treino.extend(vetor_final_imgs_saudaveis[:n_imgs_treino_sau])

  #Utilizando cerca de 20% das imagens para teste
  teste.extend(vetor_final_imgs_doentes[n_imgs_treino_doe:n_imgs_test_doe])
  teste.extend(vetor_final_imgs_saudaveis[n_imgs_treino_sau:n_imgs_test_sau])

  #Utilizando cerca de 15% das imagens para validação
  val.extend(vetor_final_imgs_doentes[n_imgs_test_doe:])
  val.extend(vetor_final_imgs_saudaveis[n_imgs_test_sau:])

  treino = np.array(treino)
  teste = np.array(teste)
  val = np.array(val)

  #LABELS --> 0 = DOENTES  ||  1 = SAUDÁVEIS
  for i in range(treino.shape[0]):
    if i < n_imgs_treino_doe:
      labels_treino.append(0)
    else:
      labels_treino.append(1)

  for i in range(teste.shape[0]):
    if i < (n_imgs_test_doe - n_imgs_treino_doe):
      labels_teste.append(0)
    else:
      labels_teste.append(1)

  for i in range(val.shape[0]):
    if i < (len(val) - n_imgs_test_doe):
      labels_val.append(0)
    else:
      labels_val.append(1)



  print(f"Labels de Treino: {labels_treino}")
  print(f"Labels de Teste: {labels_teste}")
  print(f"Labels de Validação: {labels_val}")
  
  labels_treino = np.array(labels_treino)
  labels_teste = np.array(labels_teste)
  labels_val = np.array(labels_val)


  print(treino.shape)
  print(teste.shape)
  print(val.shape)
  
  classifier(treino, teste, val, labels_treino, labels_teste, labels_val)


def classifier(train_set, test_set, val_set, train_label_set, test_label_set, val_label_set):

  model = Sequential([
      TCN(nb_filters = 64, dropout_rate=0.2, kernel_initializer="glorot_uniform",use_skip_connections=True),
      tf.keras.layers.Dense(units=1,activation="sigmoid")
  ])

  model.compile(optimizer="adam", loss = tf.losses.BinaryCrossentropy(), metrics = ["accuracy"])
  print("\n\n")
  
  hist = model.fit(train_set, train_label_set, validation_data=(val_set, val_label_set), epochs=100)

  model.summary()

  yhat = model.predict(val_set)
  print(f"Prediction Shape: {yhat}")
